In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

np.set_printoptions(precision=4, suppress=True)


In [13]:
def compute_derivatives(pos, dt):
    vel = np.gradient(pos, axis=1) / dt
    acc = np.gradient(vel, axis=1) / dt
    return vel, acc

def resample_traj(t, pos, T=200, kind='linear'):
    t = np.asarray(t).reshape(-1)
    t0, t1 = float(t[0]), float(t[-1])
    t_rs = np.linspace(t0, t1, T)
    pos_rs = np.zeros((pos.shape[0], T))
    for d in range(pos.shape[0]):
        f = interp1d(t, pos[d], kind=kind, fill_value="extrapolate", assume_sorted=True)
        pos_rs[d] = f(t_rs)
    dt = (t1 - t0) / (T - 1) if T > 1 else 1.0
    return t_rs, pos_rs, dt

def smooth_traj(pos, window=11, poly=3):
    if window >= pos.shape[1] or window % 2 == 0:  # window 需为奇数
        return pos.copy()
    pos_s = np.zeros_like(pos)
    for d in range(pos.shape[0]):
        pos_s[d] = savgol_filter(pos[d], window_length=window, polyorder=poly, mode='interp')
    return pos_s

def normalize_space(pos, mode='zscore'):
    stats = {}
    if mode == 'zscore':
        mean = pos.mean(axis=1, keepdims=True)
        std  = pos.std(axis=1, keepdims=True) + 1e-8
        pos_n = (pos - mean) / std
        stats = {'mode':'zscore','mean':mean,'std':std}
    elif mode == 'minmax':
        mn = pos.min(axis=1, keepdims=True)
        mx = pos.max(axis=1, keepdims=True)
        rng = (mx - mn) + 1e-8
        pos_n = (pos - mn) / rng * 2.0 - 1.0
        stats = {'mode':'minmax','min':mn,'max':mx}
    else:
        pos_n = pos.copy()
        stats = {'mode':'none'}
    return pos_n, stats

def denormalize_space(pos_n, stats):
    mode = stats.get('mode','none')
    if mode == 'zscore':
        return pos_n * stats['std'] + stats['mean']
    elif mode == 'minmax':
        return (pos_n + 1.0) * (stats['max'] - stats['min']) / 2.0 + stats['min']
    else:
        return pos_n

def preprocess_traj_from_demos(pos, dt=0.005, T=200,
                               smooth=True, sg_window=11, sg_poly=3,
                               space_norm='zscore'):
    """
    处理 A/B/G/H.mat 的原始演示数据：
    输入 pos: (2,L)
    """
    D, L = pos.shape
    t = np.linspace(0.0, dt*(L-1), L)
    t_rs, pos_rs, dt_rs = resample_traj(t, pos, T=T, kind='linear')
    pos_sm = smooth_traj(pos_rs, window=sg_window, poly=sg_poly) if smooth else pos_rs
    pos_n, stats = normalize_space(pos_sm, mode=space_norm)
    vel_n, acc_n = compute_derivatives(pos_n, dt_rs)
    return {
        't': t_rs, 'dt': dt_rs,
        'pos': pos_n, 'vel': vel_n, 'acc': acc_n,
        'pos_raw': pos_rs, 'stats': stats
    }


In [15]:
# 配置
data_file = '../2Dletters/G.mat'  # 可改成 '../2Dletters/H.mat' 等
demo_idx  = 0                     # 第几条演示（0~9）

# 读取与解包
mat = loadmat(data_file)
demos = mat['demos']                         # 1x10 结构数组
pos_raw = demos[0, demo_idx]['pos']          # 原始 pos（object/嵌套）
pos = extract_pos_2xT(pos_raw)               # -> (2, T)

# 统一预处理
prep = preprocess_traj_from_demos(
    pos, dt=0.005, T=200,
    smooth=True, sg_window=11, sg_poly=3,
    space_norm='zscore'
)

t_rs   = prep['t']
pos_n  = prep['pos']   # 归一化后位置 -> 用于训练 DMP
vel_n  = prep['vel']
acc_n  = prep['acc']
stats  = prep['stats']

print("pos_n shape:", pos_n.shape, "dt:", prep['dt'])


ValueError: No axis of size 2 in pos shape ()

In [6]:
# 修改为你的数据相对路径
mat = loadmat('../2Dletters/G.mat')  # 或 '../2Dletters/H.mat' 等
demos = mat['demos']                  # 1x10 结构数组
pos = demos[0,0]['pos']               # (2, L) -> [x; y]

prep = preprocess_traj_from_demos(
    pos, dt=0.005, T=200,
    smooth=True, sg_window=11, sg_poly=3,
    space_norm='zscore'
)

t_rs   = prep['t']
pos_n  = prep['pos']   # 归一化后位置 -> 用于训练 DMP
vel_n  = prep['vel']
acc_n  = prep['acc']
stats  = prep['stats'] # 反归一化要用


ValueError: setting an array element with a sequence.